In [2]:
!pip install os
!pip install json
!pip install math
!pip install torch
!pip install transformers
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement math (from versions: none)
ERROR: No matching distribution found for math
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install tf-keras

Defaulting to user installation because normal site-packages is not writeable
  Using cached tf_keras-2.19.0-py3-none-any.whl (1.7 MB)
  Using cached tensorflow-2.19.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (644.8 MB)
  Using cached numpy-2.1.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4


In [4]:
!pip install transformers[torch]

Defaulting to user installation because normal site-packages is not writeable


In [5]:
!pip install 'accelerate>=0.26.0'

Defaulting to user installation because normal site-packages is not writeable


In [6]:
!pip uninstall keras tensorflow tf-keras -y

Found existing installation: keras 3.6.0
Not uninstalling keras at /usr/lib/python3/dist-packages, outside environment /usr
Can't uninstall 'keras'. No files were found to uninstall.
Found existing installation: tensorflow 2.19.0
Uninstalling tensorflow-2.19.0:
  Successfully uninstalled tensorflow-2.19.0
Found existing installation: tf_keras 2.19.0
Uninstalling tf_keras-2.19.0:
  Successfully uninstalled tf_keras-2.19.0


In [7]:
!pip install torch torchvision
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [8]:
!pip install accelerate

Defaulting to user installation because normal site-packages is not writeable


In [9]:
!pip uninstall numpy -y

Found existing installation: numpy 2.1.3
Uninstalling numpy-2.1.3:
  Successfully uninstalled numpy-2.1.3


In [10]:
!pip install numpy==1.24.4

Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-1.24.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)


In [11]:
!pip install "numpy<2.0"

Defaulting to user installation because normal site-packages is not writeable


In [13]:
!pip uninstall keras
!pip install keras==2.11.0
!pip install tensorflow==2.11.0

Found existing installation: keras 3.6.0
Not uninstalling keras at /usr/lib/python3/dist-packages, outside environment /usr
Can't uninstall 'keras'. No files were found to uninstall.
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.1 MB/s eta 0:00:00a 0:00:01
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 4.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 84.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 KB 82.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 KB 91.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 KB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 64.9 MB/s eta

In [17]:
!pip install tf-keras

Defaulting to user installation because normal site-packages is not writeable


In [18]:
import os
import json
import math
import keras
import tensorflow
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    TrainerCallback,
)
from datasets import Dataset

def load_squad_finetune(path):
    with open(path) as f:
        squad = json.load(f)["data"]

    samples = []
    for article in squad:
        for paragraph in article["paragraphs"]:
            context = paragraph["context"]
            for qa in paragraph["qas"]:
                if qa.get("is_impossible", False):
                    continue

                question = qa["question"]
                answers = qa.get("answers", [])
                answer = answers[0]["text"] if answers else ""

                text = f"Question: {question}\nContext: {context}\nAnswer: {answer}"
                samples.append({"text": text})
    return samples

def tokenize_function(example, tokenizer):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

class LossLoggerCallback(TrainerCallback):
    def __init__(self, log_dir="metrics_logs"):
        self.epoch_metrics = {}
        os.makedirs(log_dir, exist_ok=True)
        self.log_file_path = os.path.join(log_dir, "gpt2_vanilla_finetuning_squad.txt")

    def on_epoch_end(self, args, state, control, **kwargs):
        epoch = int(state.epoch)
        log_history = state.log_history[-1]

        clm_loss = log_history.get("loss", None)
        learning_rate = log_history.get("learning_rate", None)

        perplexity = math.exp(clm_loss) if clm_loss is not None else None

        total_tokens = (
            args.per_device_train_batch_size
            * args.gradient_accumulation_steps
            * args.logging_steps
            * 512
        )

        self.epoch_metrics[epoch] = {
            "loss": clm_loss,
            "learning_rate": learning_rate,
            "perplexity": perplexity,
            "token_count": total_tokens,
        }

        print(f">>> Epoch {epoch} Summary:")
        print(f"   Loss: {clm_loss:.4f}" if clm_loss else "   Loss: N/A")
        print(f"   Perplexity: {perplexity:.2f}" if perplexity else "   Perplexity: N/A")
        print(f"   Learning Rate: {learning_rate:.6f}" if learning_rate else "   Learning Rate: N/A")
        print(f"   Token Count (approx.): {total_tokens}")

        with open(self.log_file_path, "a") as f:
            f.write(f"Epoch {epoch}:\n")
            f.write(f"  Loss: {clm_loss:.4f}\n" if clm_loss else "  Loss: N/A\n")
            f.write(f"  Perplexity: {perplexity:.2f}\n" if perplexity else "  Perplexity: N/A\n")
            f.write(f"  Learning Rate: {learning_rate:.6f}\n" if learning_rate else "  Learning Rate: N/A\n")
            f.write(f"  Token Count: {total_tokens}\n\n")

def main():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")

    model_name = "gpt2"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(model_name)
    model.resize_token_embeddings(len(tokenizer))

    samples = load_squad_finetune("data/train-v2.0.json")
    dataset = Dataset.from_list(samples)
    dataset = dataset.map(lambda x: tokenize_function(x, tokenizer), batched=True)

    output_dir = "models/gpt2-vanilla-finetune-squad"
    os.makedirs(output_dir, exist_ok=True)

    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=100,
        num_train_epochs=5,
        save_steps=500,
        logging_steps=100,
        save_total_limit=2,
        fp16=torch.cuda.is_available(),
        logging_dir="./logs",
        report_to="none"
    )

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False
    )

    loss_logger = LossLoggerCallback()

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        callbacks=[loss_logger]
    )

    trainer.train()

    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    print(f"Vanilla fine-tuned model saved to '{output_dir}/'")
    print("Metrics per epoch:", loss_logger.epoch_metrics)

if __name__ == "__main__":
    main()


Using device: cuda


Map: 100%|██████████| 86821/86821 [00:18<00:00, 4806.83 examples/s]
/tmp/ipykernel_4068/4225814412.py:118: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


OutOfMemoryError: CUDA out of memory. Tried to allocate 600.00 MiB. GPU 0 has a total capacity of 39.49 GiB of which 169.38 MiB is free. Including non-PyTorch memory, this process has 39.32 GiB memory in use. Of the allocated memory 38.77 GiB is allocated by PyTorch, and 53.40 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)